In [66]:
import pandas as pd
import numpy as np

df = pd.read_csv('processed_parkinson_data_without_healthy_control_created_13.04.21.csv', sep=',') 
df

,PATNO,cogstate,age_cat,NHY,rigidity,upsit,tremor,gen,race,updrs_totscore,...,NP1APAT,NP1DDS,NP1HALL,DOMSIDE,sft,stai,cogstate_missed,DOMSIDE_MISSED,scopa_missed,disease_status
0,3001,1.0,2,1.0,2.0,25.0,2.0,1,1,22.0,...,0.0,0.0,0.0,2.0,42.0,51.0,1,0,0,Evolved
1,3002,1.0,3,2.0,0.0,17.0,1.0,2,1,40.0,...,1.0,0.0,0.0,1.0,62.0,69.0,1,0,0,Not Evolved
2,3003,1.0,2,2.0,7.0,23.0,3.0,2,1,47.0,...,0.0,0.0,0.0,2.0,47.0,51.0,1,0,0,Evolved
3,3006,1.0,2,2.0,3.0,13.0,0.0,2,1,29.0,...,0.0,0.0,0.0,2.0,66.0,49.0,1,0,0,Not Evolved
4,3007,1.0,2,2.0,2.0,18.0,5.0,1,1,35.0,...,0.0,0.0,0.0,2.0,41.0,60.0,1,0,0,Not Evolved
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,4125,1.0,2,2.0,4.0,26.0,10.0,1,1,44.0,...,0.0,0.0,0.0,1.0,37.0,46.0,0,0,0,Not Evolved
483,4126,1.0,1,2.0,5.0,18.0,5.0,1,1,39.0,...,2.0,0.0,0.0,2.0,57.0,87.0,0,0,0,Not Evolved
484,4127,1.0,3,2.0,10.0,30.0,5.0,1,1,35.0,...,0.0,0.0,0.0,1.0,40.0,59.0,0,0,0,Not Evolved
485,4135,2.0,3,1.0,2.0,34.0,2.0,1,1,42.0,...,2.0,0.0,0.0,2.0,38.0,68.0,0,0,0,Evolved


In [67]:
def calc_rule_metrics(rule):
    dfrule = df.copy()
    dfrule.query(rule, inplace = True)
    
    confidenceSample = df[df['disease_status']=='Evolved']['disease_status'].count() / df['disease_status'].count()

    #Support <-> Coverage <-> Cobertura
    allRuleTarget = dfrule[dfrule['disease_status'] == 'Evolved']['disease_status'].count()
    coverageRule = allRuleTarget/(df['disease_status'].count()*0.7) 
    coverageRuleTarget = allRuleTarget/df[df['disease_status']=='Evolved']['disease_status'].count()

    #Confidence <-> Confiança
    allRuleTarget = dfrule[dfrule['disease_status'] == 'Evolved']['disease_status'].count()
    allRuleTarget
    confidenceRule = allRuleTarget/dfrule['disease_status'].count()

    liftRule = confidenceRule / confidenceSample

    print("Rule:", rule,  "| Coverage: ", coverageRule ,  "| Coverage Target Class: ", coverageRuleTarget , "| Confidence: ", confidenceRule, "| Lift: ", liftRule," | Qtd El Recovered:", allRuleTarget, "\n")

In [68]:
calc_rule_metrics('NHY <= 1 and moca <= 26 and age_cat >= 3 and updrs_totscore >= 17')
calc_rule_metrics('NHY == 1 and scopa<=2 and tremor>=3')
calc_rule_metrics('NHY <= 1 and tremor>=2 and sft>=63 and upsit<=33')

#Training Set
calc_rule_metrics('updrs_totscore <= 26 and upsit<=19 and scopa>=6 and gen == 1 and rigidity>=1') 
calc_rule_metrics('gen != 1 and stai <= 55.0 and scopa >= 12.0 and rigidity >= 1.0 and stai >= 46.0 ')
calc_rule_metrics('scopa<=2.0 and updrs_totscore<=22.0 and updrs_totscore>=14.0 and scopa>=2.0')

#Full Database
calc_rule_metrics('updrs_totscore <= 26.0 and tremor>=8.0 and age_cat>=2.0 and gds_cat==0.0')
calc_rule_metrics('gen!=1 and stai<=55.0 and scopa>=12.0 and rigidity>=1.0 and stai>=46.0')
calc_rule_metrics('scopa<= 1.0 and updrs_totscore<=22.0 and tremor>=1.0 and upsit>=22.0 ')

#calc_rule_metrics('race >=4 and tremor>=5.0 and sft>=46.0') 

Rule: NHY <= 1 and moca <= 26 and age_cat >= 3 and updrs_totscore >= 17 | Coverage:  0.05866823115283075 | Coverage Target Class:  0.10204081632653061 | Confidence:  0.9090909090909091 | Lift:  2.2588126159554727  | Qtd El Recovered: 20 

Rule: NHY == 1 and scopa<=2 and tremor>=3 | Coverage:  0.04986799647990613 | Coverage Target Class:  0.08673469387755102 | Confidence:  0.8947368421052632 | Lift:  2.223147153598281  | Qtd El Recovered: 17 

Rule: NHY <= 1 and tremor>=2 and sft>=63 and upsit<=33 | Coverage:  0.0469345849222646 | Coverage Target Class:  0.08163265306122448 | Confidence:  1.0 | Lift:  2.4846938775510203  | Qtd El Recovered: 16 

Rule: updrs_totscore <= 26 and upsit<=19 and scopa>=6 and gen == 1 and rigidity>=1 | Coverage:  0.05866823115283075 | Coverage Target Class:  0.10204081632653061 | Confidence:  0.8 | Lift:  1.9877551020408164  | Qtd El Recovered: 20 

Rule: gen != 1 and stai <= 55.0 and scopa >= 12.0 and rigidity >= 1.0 and stai >= 46.0  | Coverage:  0.026400704